# Set Up NVFLARE in POC Mode

This tutorial covers in depth the available features of the POC mode. For more basic setup guides, refer to the [Quickstart](https://nvflare.readthedocs.io/en/main/quickstart.html#setting-up-the-application-environment-in-poc-mode) or [POC Mode Documentation](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/poc_command.html).

POC mode allows users to test the features of a full FLARE deployment on a single machine, without the overhead of a true distributed deployment.
Compared to the FL Simulator, where the job run is automated on a single system, POC mode allows you to establish and connect distinct server and client "systems" which can then be orchestrated using the FLARE Console. Users can also experiment with various deployment options (project.yml), making POC mode a useful tool in preparation for a distributed deployment.

>It is ideal to start your NVFLARE system in POC mode from a **terminal**, not from a notebook. The terminal's virtual environment must match the kernel's virtual environment. In our case, we are using 'nvflare_example'.

To get started, let's look at the NVFlare CLI usage for the ``poc`` subcommand:

In [ ]:
! nvflare poc -h

We can further look at the help description for each command

> We will need ```tree``` command, so we will install ```tree``` in linux, if tree is not available to you, you can replace it with 'ls -al`

In [ ]:
! python -m pip install tree

## Configure the POC Workspace

### Config command

The config command can be used to configure the `poc_workspace`, as well as other locations such as the `startup_kit` and `job_template` directories.

In [ ]:
! nvflare config -h

To specify the POC workspace, use the `--poc_workspace_dir` or `-pw` option:

In [ ]:
! nvflare config -pw /tmp/nvflare/poc --job_templates_dir ../../job_templates

### Hidden Config

The config command works by writing the POC workspace directory to a hidden configuration file, which can also be edited directly:

In [ ]:
! cat ~/.nvflare/config.conf

### Environment Variable

You can also use the env. variable NVFLARE_POC_WORKSPACE if you prefer. If both the hidden config file and env. variable NVFLARE_POC_WORKSPACE are set, the env. variable will take the precedence.

In [ ]:
%env  NVFLARE_POC_WORKSPACE=/tmp/nvflare/poc
! echo $NVFLARE_POC_WORKSPACE 

## Prepare the POC workspace

Now that we've configured our POC configuration, we can prepare the POC workspace.  By default, the following command will generate POC packages for a server and two clients. If there is a pre-existing POC workspace, you will be prompted whether to overwrite and create a new one.

Below we explore the various prepare options:

In [ ]:
!  nvflare poc prepare -h

### Number of Clients

Specify a different number of clients with -n:

```
nvflare poc prepare -n [NUMBER_OF_CLIENTS]
```

In [ ]:
! echo 'y' | nvflare poc prepare -n 2

### Named Clients

Specifiy the client site names ((instead of use default site-1,2 etc.) with -c:

```
nvflare poc prepare -c [CLIENTS ...]
```

In [ ]:
! echo 'y' | nvflare poc prepare -c hospital-1 hospital-2

In [ ]:
! tree /tmp/nvflare/poc/example_project/prod_00

### Docker Image

The -d option instructs the provision tool to create a docker.sh use the specified docker_image. The docker.sh will start in detached mode. 

```
nvflare poc prepare -d [DOCKER_IMAGE]
```

This will create docker.sh for each client and server which will run `docker pull docker_image` from docker hub, then running the docker in detached mode. The docker_image must use nvflare. For example, we use the following docker image "nvflare/nvflare" which has nvflare preinstalled.
> Note: you should build your own image if you want to include your dependencies such as pytorch etc.


In [ ]:
! echo 'y'| nvflare poc prepare -d 'nvflare/nvflare'

In [ ]:
! tree /tmp/nvflare/poc/example_project/prod_00

Notice the new "docker.sh", now each client will using docker run with we specified "nvflare/nvflare" docker image

### Homomorphic Encryption (HE)

The -he option enables homomorphic encryption.

```
nvflare poc prepare -he
```

To support HE, we need the provision process to generate TenSEAL homomorphic encryption context for the server and client and write them to the server and client participant folders (See [provision context](https://nvflare.readthedocs.io/en/main/programming_guide/provisioning_system.html#provision-context)). This is achieved by the HEBuilder, which is automatically added to the `project.yml` file using the `-he` in the POC command (See the `-i` option to manually customize the `project.yml` file).

First we must install nvflare with TenSEAL:

> note: TenSEAL is not available in Mac OS

In [ ]:
! pip install nvflare[HE]

For an example, we can prepare a POC workspace with HE enabled with the following command:

In [ ]:
! echo 'y' | nvflare poc prepare -he

In [ ]:
! cat  /tmp/nvflare/poc/project.yml

Notice:  ```path: nvflare.lighter.impl.he.HEBuilder``` is added to the YAML file.
Also note the importance of the order of the builders, as the `HEBuilder` must be configured before the `SignatureBuilder` to ensure the tenseal context files are signed.

### Project Input: Simulate production deployment setup locally

Assuming we have a new custom project.yml, we can test out the project.yml locally in POC mode before we do the real provisionining.

```
nvflare poc prepare -i [PROJECT_INPUT]
```

Let's try this out. We have prepared an custom_project.yml file: 

In [ ]:
! cat ./custom_project.yml

Notice in this project file, we define three sites: 

* nonprofit-server, served as Server,belong to nonprofit organizaton 
* us_hospital, served as client, belong to us-hospital
* europe-hospital, served as client, belong to eu-hospital

We also define few users: 
* admin@nonprofit.org, from nonprofit.org, serves as project admin
* admin@hospital.org.us, lead@hospital.org.us,  member@hospital.org.us from us_hospital, serves as org admin, lead and member for us_hospital respectively
* admin@hospital.org.eu, lead@hospital.org.eu,  member@hospital.org.eu from eu_hospital, serves as org admin, lead and member for eu_hospital respectively

In [ ]:
! echo 'y' | nvflare poc prepare -i ./custom_project.yml

In [ ]:
! tree /tmp/nvflare/poc/health_project/prod_00

See that we successfully provisioned all participants from our "custom_project.yml".

## Prepare job example directories

By default, the `nvflare poc prepare` command will setup a symbolic link to the NVFlare/examples directory, assuming that you have used `git clone` to download  NVFlare from github and have defined the NVFLARE_HOME env. variable.
    
Rather than setting the NVFLARE_HOME env. variables, instead link to your desired jobs with the following command to setup to the job directory:
    
```
nvflare poc prepare-jobs-dir -j [JOBS_DIR]
```

For example we can set the jobs directory to NVFlare/examples: 

In [ ]:
! nvflare poc prepare-jobs-dir -j ..

## Start NVFlare system in POC mode

When starting the POC deployment, it's necessary to use a separate terminal since the `nvflare poc start` command will run  in the foreground emitting output from the server and any connected clients.

Also note that `nvflare poc start` starts all clients and server as well as project admin console. All other participants (org admin, lead and member) will not started. 

In many cases, we want to start the clients and server, but without the FLARE console for project admin, as we are going to interact with the system from a notebook or CLI. You first need to find out the user name for the project admin. By default, this is admin@nvidia.com

Create a terminal to execute the following command (in JupyterLab or host terminal):

```
    nvflare poc start -ex admin@nvidia.com
```
>**Note:**
    Using ```%%bash -bg``` to run the above command in a code cell may not always work
    
### A More Complex Example
    
In this example, lets see how to start the individual user participants defined in custom_project.yml. Note the project admin is "admin@nonprofit.org" instead of "admin@nvidia.com"

Please use terminal to try these commands:

1. We start the system with two hospital clients and server, without the project admin console:

```
    nvflare poc start -ex admin@nonprofit.org
```

2. Next we like to start US hospital's lead's FLARE console:

```
    nvflare poc start -p lead@hospital.org.us
```

Once the system is up, we are now ready to go back to the notebook to work on your project.

For more details on how to operate the FLARE system and submit jobs, refer to the [POC Mode Documentation](https://nvflare.readthedocs.io/en/main/user_guide/nvflare_cli/poc_command.html#operating-the-system-and-submitting-a-job).

## Stop NVFlare system in POC mode

Once you are done with the NVFlare system, you can shut down the poc system.
From terminal, issue the following command to shutdown all participants including server, clients, and admin consoles.
```
    nvflare poc stop
```

If you like to shutdown an invididual participant, use the following -p option:

```
    nvflare poc stop -p [SERVICE]
```

Some examples of service participant names can be "server", "site-1", "admin@nvidia.com", or "lead@hospital.org.us" depending on your prepared POC. 

## Project Yaml File

Once we experimented with different options, we are ready to move on to the production beyond local host.
The same project.yml file can be used for production, and is auto-generated for you and located at:

```
    <poc_workspace>/project.yml
```

For example,


In [ ]:
! echo 'y' | nvflare poc prepare -c hospital_1 hospital_2 -d 'nvflare/nvflare'

In [ ]:
! tree /tmp/nvflare/poc/example_project/prod_00

In [ ]:
! cat  /tmp/nvflare/poc/project.yml

Copy this file and you are ready to go !

Learn more about how to take your project to the real world with the provisioning tool, dashboard, and various deployment options in our [documentation](https://nvflare.readthedocs.io/en/main/real_world_fl/overview.html).

## Clean up POC workspace

Lastly, if you want to clean up the POC workspace and delete the POC workspace directory containing all of the packages you may have prepared, you can use the following command:

In [ ]:
! nvflare poc clean

You can check if the nvflare system is shutdown cleanly with:

```
ps -eaf | grep nvflare
```

If you see output like the following, then nvflare systems are still running:

```
510535    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-1/startup/.. -s fed_client.json --set secure_train=true uid=site-1 org=nvidia config_folder=config
510539    1932  1 18:54 pts/1    00:00:03 python3 -u -m nvflare.private.fed.app.client.client_train -m /tmp/workspace/example_project/prod_00/site-2/startup/.. -s fed_client.json --set secure_train=true uid=site-2 org=nvidia config_folder=config
510543    1932  1 18:54 pts/1    00:00:04 python3 -u -m nvflare.private.fed.app.server.server_train -m /tmp/workspace/example_project/prod_00/localhost/startup/.. -s fed_server.json --set secure_train=true org=nvidia config_folder=config
```

If you have already used ``nvflare poc clean`` to delete the workspace, you may need to kill the processes manually.
